<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/FrozenLake_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable-baselines3==2.0.0a5
!pip install swig
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 25.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

Optimization:

# Environment:

In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

env = gym.make('FrozenLake-v1', desc=generate_random_map(size=8), is_slippery = True)

In [4]:
# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 2
Environment is reset
Action taken: 3
Action taken: 3
Action taken: 0
Action taken: 2
Environment is reset
Action taken: 1
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 1
Environment is reset
Action taken: 1
Action taken: 2


In [6]:
# Vectorize environment
vec_env = make_vec_env('FrozenLake-v1', n_envs=16)

# PPO Model

In [13]:
model = PPO(
    policy = 'MlpPolicy',
    env = vec_env,
    n_steps = 19,
    learning_rate=0.00047,
    batch_size = 64,
    n_epochs = 5,
    gamma = 0.996,
    gae_lambda = 0.8776,
    ent_coef = 0.001757,
    verbose=1).learn(50000)

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.24     |
|    ep_rew_mean     | 0.0294   |
| time/              |          |
|    fps             | 10735    |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 304      |
---------------------------------


/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:148: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 304`, after every 4 untruncated mini-batches, there will be a truncated mini-batch of size 48
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=19 and n_envs=16)
  warnings.warn(


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.71          |
|    ep_rew_mean          | 0.0145        |
| time/                   |               |
|    fps                  | 2988          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 608           |
| train/                  |               |
|    approx_kl            | 0.00072643626 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.39         |
|    explained_variance   | -1.3          |
|    learning_rate        | 0.00047       |
|    loss                 | 0.00337       |
|    n_updates            | 5             |
|    policy_gradient_loss | -0.00403      |
|    value_loss           | 0.0279        |
-------------------------------------------
------------------------------------------
| rollout/                |      

In [9]:
!pip install optuna
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.5 MB/s eta 0:00:00


In [11]:
import optuna
import numpy as np
from stable_baselines3.common.evaluation import evaluate_policy

def objective(trial):

    # Suggest hyperparameters
    n_steps = trial.suggest_int('n_steps', 10, 2048, log=True)
    n_epochs = trial.suggest_int('n_epochs', 3, 10)
    gamma = trial.suggest_loguniform('gamma', 0.8, 0.9999)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    ent_coef = trial.suggest_loguniform('ent_coef', 0.00001, 0.1)
    gae_lambda = trial.suggest_uniform('gae_lambda', 0.8, 1.0)

    # Create the PPO model
    model = PPO('MlpPolicy', vec_env, verbose=0, gamma=gamma,
                n_steps=n_steps, ent_coef=ent_coef,
                learning_rate=learning_rate, gae_lambda=gae_lambda,
                tensorboard_log="./ppo_frozenlake_tensorboard/")

    # Train the model
    model.learn(total_timesteps=20000)

    # Evaluate the model
    mean_reward, _ = evaluate_policy(model, vec_env, n_eval_episodes=10)

    # Close the environment
    env.close()

    return mean_reward



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best trial:")
trial = study.best_trial

print(" Value: {}".format(trial.value))
print(" Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-05-02 08:07:56,458] A new study created in memory with name: no-name-52450b12-3d47-405b-8959-9701df3db926
<ipython-input-11-dd22a8f3e999>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 0.8, 0.9999)
<ipython-input-11-dd22a8f3e999>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
<ipython-input-11-dd22a8f3e999>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  ent_coef = trial.suggest_logunif

Best trial:
 Value: 0.9
 Params: 
    n_steps: 19
    n_epochs: 5
    gamma: 0.9960282545103997
    learning_rate: 0.00046974064922605227
    ent_coef: 0.001757185895722159
    gae_lambda: 0.8776126713871909


In [15]:
#Evaluate model
eval_env = Monitor(gym.make("FrozenLake-v1"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=0.60 +/- 0.48989794855663565


# Test the trained data

In [23]:
# Test the trained agent
# using the vecenv
obs = vec_env.reset()
n_steps = 100
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=True)
    print(f"Step {step + 1}")
    print("Action: ", action)
    obs, reward, done, info = vec_env.step(action)
    print("obs=", obs, "reward=", reward, "done=", done)
    vec_env.render()
    if done.all():
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print("Goal reached!", "reward=", reward)
        break

Step 1
Action:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
obs= [4 4 4 0 4 0 0 4 0 0 4 4 4 0 4 0] reward= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] done= [False False False False False False False False False False False False
 False False False False]
Step 2
Action:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
obs= [0 4 4 0 4 4 0 8 0 4 4 8 0 0 4 0] reward= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] done= [False False False False False False False False False False False False
 False False False False]
Step 3
Action:  [0 0 0 0 0 0 0 3 0 0 0 3 0 0 0 0]
obs= [4 4 0 0 4 0 4 4 0 4 0 8 0 0 0 0] reward= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] done= [False False False False False False False False False False False False
 False False False False]
Step 4
Action:  [0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
obs= [8 4 0 0 8 0 0 4 0 4 4 4 0 0 0 4] reward= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] done= [False False False False False False False False False False False False
 False False False